As linhas de código a seguir visam carregar, com pandas, um documento json para um dataframe e normalizá-lo, de forma que haverá lihas para cada registro de uma determinada coluna.

In [1]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json

In [13]:
import json

with open(file='deliveries.json', mode='r', encoding='utf8') as localfile:
  file_ = json.load(localfile)
  sample = file_[0]

display(sample.keys())

dict_keys(['name', 'region', 'origin', 'vehicle_capacity', 'deliveries'])

In [3]:
print(sample['origin']['lng']) # dois colchetes para acessar dicionário dentro de dicionário.

-48.05498915846707


In [4]:
import pandas

deliver_df = pandas.DataFrame(file_)

display(deliver_df.head(10))

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."
5,cvrp-2-df-19,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'f4b64306e4c8185fe4baf332fdfdd22e', 'p..."
6,cvrp-0-df-7,df-0,"{'lng': -47.802664728268745, 'lat': -15.657013...",180,"[{'id': '6cc90b13f3c4386580f4ca6c6799b2ed', 'p..."
7,cvrp-2-df-26,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '5d8f8d40588819594b3e28cf2993c466', 'p..."
8,cvrp-1-df-5,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': '66ccd3ae2e54b3446f5a230996394f1d', 'p..."
9,cvrp-0-df-74,df-0,"{'lng': -47.802664728268745, 'lat': -15.657013...",180,"[{'id': '44a85635558f376bb4987e18b9c8dc7', 'po..."


- Já que tem dicionário dentro de dicionário, para separar *lat* e *lng* dentro *origin*, é preciso fazer o processo de *flatten*, usando um método de "normalização" do Pandas para ler formamto json.

In [5]:
deliver_origin_df = pandas.json_normalize(deliver_df['origin'])

display(deliver_origin_df.head(10))

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145
5,-48.054989,-15.838145
6,-47.802665,-15.657014
7,-48.054989,-15.838145
8,-47.893662,-15.805118
9,-47.802665,-15.657014


- Tendo organizado os dados dentro de origin em uma pequena tabela, agora se usa o método **merge** para unir a tabela menor à tabela principal.

In [6]:
deliveries_df = pandas.merge(left=deliver_df, right=deliver_origin_df, how='inner', left_index=True, right_index=True) # vai colocar o novo df do lado direito do antigo, e alinhar os índices (0 com 0, 1 com 1, etc)
deliveries_df = deliveries_df.drop('origin', axis=1) # excluindo a coluna 'origem', já que não há mais conteúdo nela.
deliveries_df = deliveries_df[['name','region', 'lat', 'lng', 'vehicle_capacity', 'deliveries']] # reordenando as colunas. Lembrar do duplo colchete.

deliveries_df.rename(columns={'lng':'long_hub', 'lat':'lati_hub'}, inplace=True) # por algum motivo o rename não funciona se tiver com uma variável recebendo ela. Tem que fazer só o comando.

display(deliveries_df.head(10))

,name,region,lati_hub,long_hub,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-15.838145,-48.054989,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-15.838145,-48.054989,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-15.805118,-47.893662,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-15.838145,-48.054989,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."
5,cvrp-2-df-19,df-2,-15.838145,-48.054989,180,"[{'id': 'f4b64306e4c8185fe4baf332fdfdd22e', 'p..."
6,cvrp-0-df-7,df-0,-15.657014,-47.802665,180,"[{'id': '6cc90b13f3c4386580f4ca6c6799b2ed', 'p..."
7,cvrp-2-df-26,df-2,-15.838145,-48.054989,180,"[{'id': '5d8f8d40588819594b3e28cf2993c466', 'p..."
8,cvrp-1-df-5,df-1,-15.805118,-47.893662,180,"[{'id': '66ccd3ae2e54b3446f5a230996394f1d', 'p..."
9,cvrp-0-df-74,df-0,-15.657014,-47.802665,180,"[{'id': '44a85635558f376bb4987e18b9c8dc7', 'po..."


- Chegou a vez de normalizar a coluna *deliveries*, que, diferente da coluna *origin*, onde os índices coincidiam, aquela gera muito mais elementos, tendo que ser tratada com o comando **Explode**, e ter seus índices observados.

In [7]:
deliver_exploded = deliveries_df[["deliveries"]].explode("deliveries") # o explode tem que ser feito no df original, aquele antes do merge, ou não pegará o dicionário completo.

display(deliver_exploded.head(10))

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."
0,"{'id': '71271df40c3188cda88266f9969a5a9', 'poi..."
0,"{'id': 'be0cd4bee278b16f3473c8fb05e96464', 'po..."
0,"{'id': 'e0fffb7150a2a202adb36e3f2fd5cc96', 'po..."
0,"{'id': '489efb8015b9b9e208d33abae808df23', 'po..."
0,"{'id': 'f4245aa5a08f39b615cfd57a117325ec', 'po..."


- O próximo passo é usar o comando de concatenar do pandas, com **lambda**, mudar o nome das colunas.

In [8]:
deliver_normal = pandas.concat([ # encolve tudo em um colchete só, dentro do concat.
  pandas.DataFrame(deliver_exploded["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"}), # criar um df com a coluna 'deliveries' 3 times
  pandas.DataFrame(deliver_exploded["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pandas.DataFrame(deliver_exploded["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)

display(deliver_normal.head())

,delivery_size,delivery_lng,delivery_lat
0,9,-48.116189,-15.848929
0,2,-48.118195,-15.850772
0,1,-48.112483,-15.847871
0,2,-48.118023,-15.846471
0,7,-48.114898,-15.858055


- Agora o mesmo procedimento de finalização feito no *origin* vai ser feito no *deliveries*, descartando a coluna deliveries, e usando **merge** para unir a nova ta bela à antiga.

In [11]:
deliveries_df = deliveries_df.drop('deliveries', axis=1)
deliver_df = pandas.merge(left=deliveries_df, right=deliver_normal, how='right', left_index=True, right_index=True)

display(deliver_df.head(10))

,name,region,lati_hub,long_hub,vehicle_capacity,delivery_size,delivery_lng,delivery_lat
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,9,-48.116189,-15.848929
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,2,-48.118195,-15.850772
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,1,-48.112483,-15.847871
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,2,-48.118023,-15.846471
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,7,-48.114898,-15.858055
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,10,-48.120448,-15.847780
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,7,-48.121802,-15.852089
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,10,-48.116300,-15.851215
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,9,-48.116317,-15.849660
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,8,-48.123115,-15.849925


In [12]:
deliver_df.reset_index(inplace=True, drop=True) # reset index, salving the changes with inplace and dropping

display(deliver_df.head(10))

,name,region,lati_hub,long_hub,vehicle_capacity,delivery_size,delivery_lng,delivery_lat
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,9,-48.116189,-15.848929
1,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,2,-48.118195,-15.850772
2,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,1,-48.112483,-15.847871
3,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,2,-48.118023,-15.846471
4,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,7,-48.114898,-15.858055
5,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,10,-48.120448,-15.847780
6,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,7,-48.121802,-15.852089
7,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,10,-48.116300,-15.851215
8,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,9,-48.116317,-15.849660
9,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,8,-48.123115,-15.849925
